In [19]:
import numpy as np
import pandas as pd
from scipy import linalg
from scipy import stats

np.set_printoptions(suppress=True)
np.seterr(divide='raise', invalid='raise', over='warn', under='ignore')

{'divide': 'raise', 'invalid': 'raise', 'over': 'warn', 'under': 'ignore'}

In [14]:
BETA_0 = -2
BETA_1 = np.log(2)

In [28]:
def generate_data(b, n):
    x = stats.norm.rvs(loc=0, scale=1, size=n)    
    mu = np.exp(x*BETA_1 + BETA_0)
    theta = stats.gamma.rvs(b, scale=1/b, size=n)
    return x, stats.poisson.rvs(theta*mu, size=n)

def estimate_by_poisson_model(x, y):
    def score(beta):
        beta_0 = beta[0]
        beta_1 = beta[1]
        return np.array([
            np.sum(y - np.exp(beta_0 + beta_1*x)),
            np.sum(x*y - x*np.exp(beta_0 + beta_1*x)),
        ])
    
    beta_hat = optimize.root(score, np.array([-2, np.log(2)]))['x']        
    mu_hat = np.exp(beta_hat[0] + beta_hat[1]*x)
    
    fisher_determinant = np.sum(mu_hat)*np.sum(x*x*mu_hat) - np.square(np.sum(x*mu_hat))
    beta_hat_variance = np.array([
        [np.sum(x*x*mu_hat), -np.sum(x*mu_hat)],
        [-np.sum(x*mu_hat), np.sum(mu_hat)]
    ])/fisher_determinant
    
    return beta_hat, beta_hat_variance

def estimate_by_quasi_likelihood(x, y):
    beta_hat, beta_hat_variance = estimate_by_poisson_model(x, y)
    
    mu_hat = np.exp(beta_hat[0] + beta_hat[1]*x)
    selection = mu_hat != y
    alpha_hat = np.sum(np.square(y[selection] - mu_hat[selection])/mu_hat[selection])/(len(y) - 2)
    
    return beta_hat, alpha_hat*beta_hat_variance    

def estimate_by_sandwich(x, y):
    pass

def is_covered_by_confidence_interval(estimates, actual, variances, level=0.95):
    return np.abs(estimates - actual) <= stats.norm.isf((1 - level)/2)*np.sqrt(np.diag(variances))

In [29]:
experiments = pd.DataFrame(index=pd.MultiIndex.from_product([
    #[0.2, 1, 10, 1000],
    #[10, 20, 50, 100, 250],
    [0.2], [10],
], names=['b', 'n']))

In [30]:
from scipy import optimize

np.random.seed(2018)
for b, n in experiments.index:
    print('b={}, n={}'.format(b, n))
    poisson_estimates = []
    poisson_variances = []
    
    quasi_likelihood_estimates = []
    quasi_likelihood_variances = []
    
    for i in range(10000):
        while True:
            try:
                x, y = generate_data(b, n)
                poisson_estimate, poisson_variance = estimate_by_poisson_model(x, y)
                break
            except FloatingPointError as e:
                pass
                    
        # Filter out degenerate cases
        while np.any(np.isinf(poisson_variance)) or np.any(np.diag(poisson_variance) < 0):
            x, y = generate_data(b, n)
            poisson_estimate, poisson_variance = estimate_by_poisson_model(x, y)
            
        quasi_likelihood_estimate, quasi_likelihood_variance = estimate_by_quasi_likelihood(x, y)
                    
        poisson_estimates.append(poisson_estimate)
        poisson_variances.append(poisson_variance)
        
        quasi_likelihood_estimates.append(quasi_likelihood_estimate)
        quasi_likelihood_variances.append(quasi_likelihood_variance)
        
    poisson_estimates = np.array(poisson_estimates)
    poisson_variances = np.array(poisson_variances)
    
    quasi_likelihood_estimates = np.array(quasi_likelihood_estimates)
    quasi_likelihood_variances = np.array(quasi_likelihood_variances)
    
    is_covered_by_confidence_interval_vectorized = (
        np.vectorize(
            lambda e, v: is_covered_by_confidence_interval(
                e, np.array([BETA_0, BETA_1]), v),
            otypes=[np.bool],
            signature='(i),(i,i)->(i)'))
    
    poisson_coverage = np.sum(is_covered_by_confidence_interval_vectorized(
        poisson_estimates, poisson_variances), axis=0)/len(poisson_estimates)
    
    quasi_likelihood_coverage = np.sum(is_covered_by_confidence_interval_vectorized(
        quasi_likelihood_estimates, quasi_likelihood_variances), axis=0)/len(quasi_likelihood_estimates)
    
    print(poisson_coverage)
    print(quasi_likelihood_coverage)
    
    #print(np.sum(np.abs(estimates[:,0] - BETA_0) <= np.sqrt(variances[:,0])*stats.norm.isf((1 - 0.95)/2)))
    #print(np.sum(np.abs(estimates[:,1] - BETA_1) <= np.sqrt(variances[:,1])*stats.norm.isf((1 - 0.95)/2)))    
    #print(np.mean(estimates, axis=0))

b=0.2, n=10
[0.938  0.9729]
[0.7781 0.7922]
